In [ ]:
import os
import json
import random
import logging
import shutil
import stanza
from tqdm import tqdm

In [2]:
# 假设 BASE_DIR 已经定义为项目的根目录
BASE_DIR = '/Users/jessie/Documents/Projects/Cusanus_Topic_Modeling' 

In [ ]:
# 初始化 Stanza NLP 模型
stanza.download('la')  # 下载拉丁语模型
nlp = stanza.Pipeline('la')

In [8]:
def select_documents_for_modeling(paragraphs_data_path, output_path, test_percentage=0.2):
    # 加载段落数据
    with open(paragraphs_data_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # 获取所有文档
    documents = data["documents"]
    total_documents = len(documents)

    # 检查是否有文档
    if total_documents == 0:
        print("没有找到文档，请检查 paragraphs_data.json 文件。")
        logging.error("没有找到文档，建模无法进行。")
        return []

    # 计算建模集的文档数量，确保不会超过总文档数
    modeling_document_count = max(1, min(total_documents, int(total_documents * test_percentage)))  # 至少选取一个文档用于建模

    # 随机选取建模文档
    modeling_documents = random.sample(documents, modeling_document_count)

    # 将选中的建模文档保存到新的 JSON 文件
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump({"documents": modeling_documents}, json_file, ensure_ascii=False, indent=4)

    print(f"从 {total_documents} 个文档中随机选取了 {modeling_document_count} 个完整文档并保存到 {output_path}。")
    logging.info(f"从 {total_documents} 个文档中随机选取了 {modeling_document_count} 个完整文档并保存到 {output_path}。")

    # 保存选中的文档的 txt 格式到 testset 目录
    testset_dir = os.path.join(BASE_DIR, 'data/testset')
    os.makedirs(testset_dir, exist_ok=True)

    for document in tqdm(modeling_documents, desc="复制选中的文档到 testset 目录"):
        document_id = document["document_id"]
        txt_file_name = document_id.replace('.xml', '.txt')
        source_txt_path = os.path.join(BASE_DIR, 'data/processed', txt_file_name)
        destination_txt_path = os.path.join(testset_dir, txt_file_name)

        if os.path.exists(source_txt_path):
            shutil.copy(source_txt_path, destination_txt_path)
            print(f"已将文件 {txt_file_name} 复制到 testset 目录。")
            logging.info(f"已将文件 {txt_file_name} 复制到 testset 目录。")
        else:
            print(f"未找到文件 {txt_file_name}，无法复制到 testset 目录。")
            logging.warning(f"未找到文件 {txt_file_name}，无法复制到 testset 目录。")

    # 对选中的 testset 文档进行词频、总词数、types 数和 POS frequencies 统计
    total_words = 0
    unique_words = set()
    word_freq = {}
    pos_freq = {}

    for document in tqdm(modeling_documents, desc="统计测试集文档的词频和 POS 信息"):
        for paragraph in document["paragraphs"]:
            words = paragraph["content"].split()
            total_words += len(words)
            unique_words.update(words)
            for word in words:
                word_freq[word] = word_freq.get(word, 0) + 1

            # POS tagging for the paragraph
            pos_tags = pos_tag_text(paragraph["content"])
            for _, pos in pos_tags:
                pos_freq[pos] = pos_freq.get(pos, 0) + 1

    # 计算 types 数量
    total_types = len(unique_words)

    # 打印统计结果
    print(f"测试集的统计数据：")
    print(f"总词数 (Total Words): {total_words}")
    print(f"不同词汇数量 (Total Types): {total_types}")
    print(f"词频统计 (Word Frequencies): {word_freq}")
    print(f"POS 词性分布 (POS Frequencies): {pos_freq}")

    # 保存统计结果到 JSON 文件
    testset_statistics_path = os.path.join(BASE_DIR, 'data/testset_statistics.json')
    with open(testset_statistics_path, 'w', encoding='utf-8') as json_file:
        json.dump({
            "total_words": total_words,
            "total_types": total_types,
            "word_frequencies": word_freq,
            "pos_frequencies": pos_freq
        }, json_file, ensure_ascii=False, indent=4)

    print(f"已将测试集的统计数据保存到 {testset_statistics_path}。")
    logging.info(f"已将测试集的统计数据保存到 {testset_statistics_path}。")

    return modeling_documents

In [9]:
def pos_tag_text(text):
    doc = nlp(text)
    pos_tags = [(word.text, word.upos) for sentence in doc.sentences for word in sentence.words]
    return pos_tags

In [11]:
# 定义段落数据文件路径和输出路径
paragraphs_data_path = os.path.join(BASE_DIR, 'data/processed/documents_data.json')
output_path = os.path.join(BASE_DIR, 'data/testset.json')

# %%
modeling_documents = select_documents_for_modeling(paragraphs_data_path, output_path, test_percentage=0.2)


从 306 个文档中随机选取了 61 个完整文档并保存到 /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/testset.json。


复制选中的文档到 testset 目录: 100%|██████████| 61/61 [00:00<00:00, 2814.57it/s]


已将文件 v190_291.txt 复制到 testset 目录。
已将文件 v170_055.txt 复制到 testset 目录。
已将文件 v170_081.txt 复制到 testset 目录。
已将文件 v180_193.txt 复制到 testset 目录。
已将文件 v160_020.txt 复制到 testset 目录。
已将文件 v160_006.txt 复制到 testset 目录。
已将文件 v170_027.txt 复制到 testset 目录。
已将文件 v170_093.txt 复制到 testset 目录。
已将文件 v180_138.txt 复制到 testset 目录。
已将文件 v170_070.txt 复制到 testset 目录。
已将文件 v170_054.txt 复制到 testset 目录。
已将文件 v170_106.txt 复制到 testset 目录。
已将文件 v160_018.txt 复制到 testset 目录。
已将文件 v190_275.txt 复制到 testset 目录。
已将文件 v190_226.txt 复制到 testset 目录。
已将文件 v170_080.txt 复制到 testset 目录。
已将文件 v180_133.txt 复制到 testset 目录。
已将文件 v190_268.txt 复制到 testset 目录。
已将文件 v180_134.txt 复制到 testset 目录。
已将文件 v190_250.txt 复制到 testset 目录。
已将文件 v180_192.txt 复制到 testset 目录。
已将文件 v170_074.txt 复制到 testset 目录。
已将文件 v160_015.txt 复制到 testset 目录。
已将文件 v170_061.txt 复制到 testset 目录。
已将文件 v190_216.txt 复制到 testset 目录。
已将文件 v160_010.txt 复制到 testset 目录。
已将文件 v180_198.txt 复制到 testset 目录。
已将文件 v180_167.txt 复制到 testset 目录。
已将文件 v190_247.txt 复制到 testset 目录。
已将文件 v190_252.

统计测试集文档的词频和 POS 信息: 100%|██████████| 61/61 [01:39<00:00,  1.63s/it]

测试集的统计数据：
总词数 (Total Words): 55273
不同词汇数量 (Total Types): 7020
词频统计 (Word Frequencies): {'audio': 134, 'frater': 59, 'pius': 9, 'sancio': 221, 'magnus': 229, 'pontifex': 48, 'visitatio': 8, 'primus': 261, 'urbs': 4, 'ecclesia': 108, 'committo': 23, 'placeo': 20, 'sanctitas': 13, 'impossibilis': 14, 'pareo': 12, 'saltem': 8, 'signum': 84, 'conor': 2, 'ostensio': 14, 'necessitamur': 1, 'consuetudo': 13, 'verbum': 442, 'evangelicum': 1, 'praemittamus': 1, 'divinus': 31, 'perago': 7, 'officium': 18, 'instruo': 23, 'cura': 28, 'aedifico': 7, 'deus': 1372, 'sabaoth': 2, 'agricola1': 9, 'eicio': 25, 'cultura': 1, 'textus2': 27, 'eliciamus': 2, 'pater': 334, 'familia': 22, 'figuraris': 4, 'planto': 29, 'vinea': 21, 'loco': 8, 'fructus': 78, 'via': 161, 'creator': 50, 'ascendo': 36, 'imago': 78, 'exemplar': 15, 'attendo': 76, 'simplex': 23, 'multiplex': 6, 'paternitas': 5, 'coincidit': 18, 'genitor': 2, 'filius': 427, 'rector': 17, 'senex': 12, 'honor': 36, 'debeo': 293, 'caelum': 126, 'terra': 

In [13]:
import csv
import os
import json

# 假设 BASE_DIR 已经定义为项目的根目录
BASE_DIR = '/Users/jessie/Documents/Projects/Cusanus_Topic_Modeling'
testset_statistics_path = os.path.join(BASE_DIR, 'data/testset_statistics.json')
csv_output_path = os.path.join(BASE_DIR, 'data/testset_statistics.csv')

# 加载测试集统计数据
with open(testset_statistics_path, 'r', encoding='utf-8') as json_file:
    statistics_data = json.load(json_file)

# 保存统计结果到 CSV 文件
with open(csv_output_path, 'w', encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Statistic', 'Value'])

    # 写入总词数和总类型数
    writer.writerow(['Total Words', statistics_data['total_words']])
    writer.writerow(['Total Types', statistics_data['total_types']])

    # 写入词性分布 (POS Frequencies)
    writer.writerow([])
    writer.writerow(['POS Frequencies'])
    for pos, count in statistics_data['pos_frequencies'].items():
        writer.writerow([pos, count])

    # 写入词频统计 (Word Frequencies)
    writer.writerow([])
    writer.writerow(['Word Frequencies'])
    for word, count in statistics_data['word_frequencies'].items():
        writer.writerow([word, count])

print(f"统计结果已保存到 {csv_output_path}")


统计结果已保存到 /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/testset_statistics.csv
